In [1]:
import numpy as np

# Input: expects 3xN matrix of points
# Returns R,t
# R = 3x3 rotation matrix
# t = 3x1 column vector

def rigid_transform_3D(A, B):
    assert A.shape == B.shape

    num_rows, num_cols = A.shape
    if num_rows != 3:
        raise Exception(f"matrix A is not 3xN, it is {num_rows}x{num_cols}")

    num_rows, num_cols = B.shape
    if num_rows != 3:
        raise Exception(f"matrix B is not 3xN, it is {num_rows}x{num_cols}")

    # find mean column wise
    centroid_A = np.mean(A, axis=1)
    centroid_B = np.mean(B, axis=1)

    # ensure centroids are 3x1
    centroid_A = centroid_A.reshape(-1, 1)
    centroid_B = centroid_B.reshape(-1, 1)

    # subtract mean
    Am = A - centroid_A
    Bm = B - centroid_B

    H = Am @ np.transpose(Bm)

    # sanity check
    #if linalg.matrix_rank(H) < 3:
    #    raise ValueError("rank of H = {}, expecting 3".format(linalg.matrix_rank(H)))

    # find rotation
    U, S, Vt = np.linalg.svd(H)
    R = Vt.T @ U.T

    # special reflection case
    if np.linalg.det(R) < 0:
        print("det(R) < R, reflection detected!, correcting for it ...")
        Vt[2,:] *= -1
        R = Vt.T @ U.T

    t = -R @ centroid_A + centroid_B

    return R, t

In [2]:
import pandas as pd

gcp = pd.read_csv("point_scanner_2.txt", sep=' ', index_col=0)
gcp

,x,y,z
Sphere,,,
1,-82.710228,132.319294,-143.493456
2,-88.837874,115.463071,-143.324929
3,-109.475067,137.092173,-143.150294
4,-140.819712,99.850012,-143.720015
5,-157.033742,104.765624,-144.129051
6,-169.764104,100.844141,-143.621455
7,-209.558680,104.652368,-143.763830
8,-226.399104,105.402116,-143.270310
9,-238.953785,115.976025,-143.377745


In [4]:
# sphere radius: 0.3773 ft
# pole height: 4.3229166667 ft

# adjust control points to ground
gcp.loc[:, 'z'] -= (0.3773 + 4.3229166667)
gcp_scanner = gcp.copy()
gcp_scanner

,x,y,z
Sphere,,,
1,-82.710228,132.319294,-148.193673
2,-88.837874,115.463071,-148.025146
3,-109.475067,137.092173,-147.850511
4,-140.819712,99.850012,-148.420232
5,-157.033742,104.765624,-148.829268
6,-169.764104,100.844141,-148.321672
7,-209.558680,104.652368,-148.464047
8,-226.399104,105.402116,-147.970527
9,-238.953785,115.976025,-148.077962


In [5]:
gcp_global = pd.read_csv("Hoboken_control_with_accuracy.csv", index_col=0)
gcp_global

,lon,lat,orthoheight,h_accuracy,v_accuracy
OBJECTID,,,,,
1,623205.6986,693359.2726,4.593400,0.005000,0.007
2,623198.4926,693342.7713,4.380135,0.007810,0.013
3,623179.3188,693365.8054,4.534342,0.007211,0.011
4,623145.5895,693330.6326,4.045473,0.007211,0.011
5,623129.7283,693336.5529,3.851894,0.005831,0.009
6,623116.7280,693333.5158,3.970010,0.007211,0.010
7,623077.3298,693339.8872,3.983134,0.008062,0.011
8,623060.5526,693341.7779,4.081564,0.007810,0.011
9,623048.7078,693353.1365,3.999539,0.007211,0.010


# gcp 1-6

In [6]:
A = np.array(gcp_scanner.loc[1:6, ['x', 'y', 'z']]).T
B = np.array(gcp_global.loc[1:6, ['lon', 'lat', 'orthoheight']]).T

[ret_R, ret_t] = rigid_transform_3D(A, B)

# Compare the recovered R and t with the original
B2 = (ret_R@A) + ret_t

n = A.shape[1]

# Find the root mean squared error
err = B2 - B
err = err * err
err = np.sum(err)
rmse = np.sqrt(err/n)

rmse

0.1705644380439684

# gcp 1-9

In [8]:
A = np.array(gcp_scanner.loc[1:9, ['x', 'y', 'z']]).T
B = np.array(gcp_global.loc[1:9, ['lon', 'lat', 'orthoheight']]).T

[ret_R, ret_t] = rigid_transform_3D(A, B)

# Compare the recovered R and t with the original
B2 = (ret_R@A) + ret_t

n = A.shape[1]

# Find the root mean squared error
err = B2 - B
err = err * err
err = np.sum(err)
rmse = np.sqrt(err/n)

rmse

0.16474414140249316

# gcp 10-12

In [9]:
A = np.array(gcp_scanner.loc[10:12, ['x', 'y', 'z']]).T
B = np.array(gcp_global.loc[10:12, ['lon', 'lat', 'orthoheight']]).T

[ret_R, ret_t] = rigid_transform_3D(A, B)

# Compare the recovered R and t with the original
B2 = (ret_R@A) + ret_t

n = A.shape[1]

# Find the root mean squared error
err = B2 - B
err = err * err
err = np.sum(err)
rmse = np.sqrt(err/n)

rmse

0.02236254919696122

# gcp 1-12

In [10]:
A = np.array(gcp_scanner.loc[1:12, ['x', 'y', 'z']]).T
B = np.array(gcp_global.loc[1:12, ['lon', 'lat', 'orthoheight']]).T

[ret_R, ret_t] = rigid_transform_3D(A, B)

# Compare the recovered R and t with the original
B2 = (ret_R@A) + ret_t

n = A.shape[1]

# Find the root mean squared error
err = B2 - B
err = err * err
err = np.sum(err)
rmse = np.sqrt(err/n)

rmse

0.16160300333954894

# gcp 13-15

In [11]:
A = np.array(gcp_scanner.loc[13:15, ['x', 'y', 'z']]).T
B = np.array(gcp_global.loc[13:15, ['lon', 'lat', 'orthoheight']]).T

[ret_R, ret_t] = rigid_transform_3D(A, B)

# Compare the recovered R and t with the original
B2 = (ret_R@A) + ret_t

n = A.shape[1]

# Find the root mean squared error
err = B2 - B
err = err * err
err = np.sum(err)
rmse = np.sqrt(err/n)

rmse

det(R) < R, reflection detected!, correcting for it ...


0.020970157295183225

# gcp 1-15

In [12]:
A = np.array(gcp_scanner.loc[1:15, ['x', 'y', 'z']]).T
B = np.array(gcp_global.loc[1:15, ['lon', 'lat', 'orthoheight']]).T

[ret_R, ret_t] = rigid_transform_3D(A, B)

# Compare the recovered R and t with the original
B2 = (ret_R@A) + ret_t

n = A.shape[1]

# Find the root mean squared error
err = B2 - B
err = err * err
err = np.sum(err)
rmse = np.sqrt(err/n)

rmse

0.20387722097216635

# gcp 16-18

In [13]:
A = np.array(gcp_scanner.loc[16:18, ['x', 'y', 'z']]).T
B = np.array(gcp_global.loc[16:18, ['lon', 'lat', 'orthoheight']]).T

[ret_R, ret_t] = rigid_transform_3D(A, B)

# Compare the recovered R and t with the original
B2 = (ret_R@A) + ret_t

n = A.shape[1]

# Find the root mean squared error
err = B2 - B
err = err * err
err = np.sum(err)
rmse = np.sqrt(err/n)

rmse

0.012394192030031127

# gcp 1-18

In [14]:
A = np.array(gcp_scanner.loc[1:18, ['x', 'y', 'z']]).T
B = np.array(gcp_global.loc[1:18, ['lon', 'lat', 'orthoheight']]).T

[ret_R, ret_t] = rigid_transform_3D(A, B)

# Compare the recovered R and t with the original
B2 = (ret_R@A) + ret_t

n = A.shape[1]

# Find the root mean squared error
err = B2 - B
err = err * err
err = np.sum(err)
rmse = np.sqrt(err/n)

rmse

0.24639673988350974